In [ ]:
import pyflowbat as pfb
import numpy as np

In [ ]:
my_wrkspc = pfb.pyflowbat.Workspace(full_output=True)

In [ ]:
my_wrkspc.calculate_beads_factors(
    beads_file_file_path="./ebrc-poster-example-data/Beads_After.fcs",
    beads_fluorescent_channels=[
        ("PE-Texas Red-A", "MEPTRs"),
        ("FITC-A", "MEFLs")
    ],
    beads_num_pops=9
)

In [ ]:
my_wrkspc.load_samples(
    sample_collection_name="raw",
    samples_folder_path="./ebrc-poster-example-data/",
    include=['.fcs'],
    not_include=['Beads']
)

In [ ]:
my_wrkspc.apply_gate(
    sample_collection_name='raw',
    new_sample_collection_name='heks',
    gating_function=pfb.gating.gate_heks,
    method = 'same',
    samples = ['373_C_001.fcs', '664_D_002.fcs', '373_M_001.fcs']
)

In [ ]:
i = 0
for sample in my_wrkspc.sample_collections['raw']:
    if i > 5:
        break
    print(sample)
    my_wrkspc.visualize_plot_change(
        sample_collection_name_0="raw",
        sample_collection_name_f='heks',
        sample_name=sample,
        channel_names=("FSC-A", "SSC-A")
    )
    i += 1

In [ ]:
my_wrkspc.apply_gate(
    sample_collection_name='heks',
    new_sample_collection_name='singlets',
    gating_function=pfb.gating.gate_singlets
)

In [ ]:
i = 0
for sample in my_wrkspc.sample_collections['heks']:
    if i > 5:
        break
    print(sample)
    my_wrkspc.visualize_plot_change(
        sample_collection_name_0="heks",
        sample_collection_name_f='singlets',
        sample_name=sample,
        channel_names=("FSC-A", "FSC-H")
    )
    i += 1

In [ ]:
my_wrkspc.calculate_compensation_matrix(
    sample_collection_name="singlets",
    compensation_sample_names=['Colors_DsRE2.fcs', 'Colors_mNG.fcs'],
    compensation_channel_names=['PE-Texas Red-A', 'FITC-A'],
    threshold=10**-3
)

In [ ]:
my_wrkspc.apply_compensation_matrix(
    sample_collection_name="singlets",
    new_sample_collection_name="compensated"
)

In [ ]:
my_wrkspc.apply_gate(
    sample_collection_name="compensated",
    new_sample_collection_name="transfected",
    gating_function=pfb.gating.gate_high_low,
    gating_channel_name="Alexa 750-A",
    low=pfb.gating.find_percentile(
        workspace=my_wrkspc,
        sample_collection_name="compensated",
        sample_name="Colors_Filler.fcs",
        channel_name="Alexa 750-A",
        percentile=99
    )
)

In [ ]:
dox_conc = {
    'A':0,
    'B':10**0,
    'C':10**0.17,
    'D':10**0.33,
    'E':10**0.50,
    'F':10**0.63,
    'G':10**0.75,
    'H':10**0.88,
    'I':10**1.0,
    'J':10**1.08,
    'K':10**1.25,
    'L':10**1.33,
    'M':10**1.50,
    'N':10**1.75,
    'O':10**2.0,
    'P':10**3.0
}

In [ ]:
extraction = my_wrkspc.create_statistic_extraction(
    sample_collection_name='transfected',
    statistics_collection_name='samples',
    include=['.fcs'],
    not_include=['Beads', 'Controls', 'Colors'],
    statistic_names=['line', 'dox_conc', 'sample','Mean FITC-A', 'Mean PE-Texas Red-A']
)
my_wrkspc.extract_statistic(
    extraction=extraction,
    statistc_name='line',
    operation=pfb.operations.split_sample_name,
    by = '_',
    index = 0
)
my_wrkspc.extract_statistic(
    extraction=extraction,
    statistc_name='dox_conc',
    operation=pfb.operations.split_sample_name,
    by = '_',
    index = 1
)
my_wrkspc.extract_statistic(
    extraction=extraction,
    statistc_name='sample',
    operation=pfb.operations.split_sample_name,
    by = '_',
    index = 2
)
my_wrkspc.extract_statistic(
    extraction=extraction,
    statistc_name='Mean FITC-A',
    operation=pfb.operations.channel_mean,
    channel_name = 'FITC-A' 
)
my_wrkspc.extract_statistic(
    extraction=extraction,
    statistc_name='Mean PE-Texas Red-A',
    operation=pfb.operations.channel_mean,
    channel_name = 'PE-Texas Red-A'
)

In [ ]:
my_wrkspc.combine_replicates(
    statistics_collection_name='samples',
    combined_statistics_collection_name='samples combined',
    combine_by=['dox_conc', 'line'],
    combination_operations={
        'Mean FITC-A': 'mean',
        'Mean PE-Texas Red-A': 'mean'
    },
    sem_cols=[
       'Mean FITC-A',
       'Mean PE-Texas Red-A' 
    ]
)

In [ ]:
my_wrkspc.apply_operation(
    statistics_collection_name="samples combined",
    new_statistics_collection_name="samples converted",
    statistic_name="Mean FITC-A",
    new_statistic_name="Mean FITC-A",
    operation=pfb.operations.non_negative
)
my_wrkspc.apply_operation(
    statistics_collection_name="samples combined",
    new_statistics_collection_name="samples converted",
    statistic_name="Mean PE-Texas Red-A",
    new_statistic_name="Mean PE-Texas Red-A",
    operation=pfb.operations.non_negative
)
my_wrkspc.apply_operation(
    statistics_collection_name="samples converted",
    new_statistics_collection_name="samples converted",
    statistic_name="Mean FITC-A",
    new_statistic_name="MEFLs",
    operation=pfb.operations.apply_conversion_factor,
    factor=my_wrkspc.conversion_factors["FITC-A"]
)
my_wrkspc.apply_operation(
    statistics_collection_name="samples converted",
    new_statistics_collection_name="samples converted",
    statistic_name=["Mean FITC-A", "Mean FITC-A_stdErr"],
    new_statistic_name="MEFLs_stdErr",
    operation=pfb.operations.compute_conversion_factor_stdErr,
    factor=my_wrkspc.conversion_factors["FITC-A"],
    factor_err=my_wrkspc.conversion_factors["FITC-A_stderr"]
)
my_wrkspc.apply_operation(
    statistics_collection_name="samples converted",
    new_statistics_collection_name="samples converted",
    statistic_name="Mean PE-Texas Red-A",
    new_statistic_name="MEPTRs",
    operation=pfb.operations.apply_conversion_factor,
    factor=my_wrkspc.conversion_factors["PE-Texas Red-A"]
)
my_wrkspc.apply_operation(
    statistics_collection_name="samples converted",
    new_statistics_collection_name="samples converted",
    statistic_name=["Mean PE-Texas Red-A", "Mean PE-Texas Red-A_stdErr"],
    new_statistic_name="MEPTRs_stdErr",
    operation=pfb.operations.compute_conversion_factor_stdErr,
    factor=my_wrkspc.conversion_factors["PE-Texas Red-A"],
    factor_err=my_wrkspc.conversion_factors["PE-Texas Red-A_stderr"]
)

In [ ]:
my_wrkspc.graph_statistics(
    data=[
        ["samples converted", 'MEFLs', 'MEPTRs', ('line', '373')],
        ["samples converted", 'MEFLs', 'MEPTRs', ('line', '664')],
        ["samples converted", 'MEFLs', 'MEPTRs', ('line', '665')],
        ["samples converted", 'MEFLs', 'MEPTRs', ('line', '666')],
        ["samples converted", 'MEFLs', 'MEPTRs', ('line', '667')]
    ],
    errors=[True, True],
    legend=["no synTF", "VP64", "VPR", "p65", "p65trunc2"],
    title="ZF1 DNA-binding domain:\n MEPTRs vs. MEFLs",
    labels=["synTF proxy expression (MEFLs)", "Reporter expression (MEPTRs)"],
    xlog=True,
    ylog=True,
    save=True
)

In [ ]:
my_wrkspc.graph_statistics(
    data=[
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '373']],
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '664']],
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '668']],
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '669']],
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '670']],
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '671']],
        ["samples converted", 'MEFLs', 'MEPTRs', ['line', '672']]
    ],
    errors=[True, True],
    legend=["no synTF", "ZF1", "ZF6", "ZF8", "ZF9", "ZF10", "ZF15"],
    title="VP64 activation domain:\n MEPTRs vs. MEFLs",
    labels=["synTF proxy expression (MEFLs)", "Reporter expression (MEPTRs)"],
    xlog=True,
    ylog=True,
    save="tst.png"
)

In [ ]:
my_wrkspc.stats_collections['samples converted'].to_csv("ebrc-example-output.csv")